In [1]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
# import pandas as pd

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('C:/Users/prath/Downloads/dogs-vs-cats/sampleSubmission.csv')

### Preprocessing 

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_augmentation = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

### Creating batches of size 128 with target size 128 by 128 from training folder

In [8]:
train_generator = train_augmentation.flow_from_directory(
    directory = r'.//dogs-vs-cats//train',
    target_size = (128, 128),
    batch_size = 128,
    class_mode = 'binary'
)

Found 0 images belonging to 0 classes.


In [9]:
validation_augmentation = ImageDataGenerator(
        rescale = 1./255
)

In [11]:
validation_generator = validation_augmentation.flow_from_directory(
    directory = './Testing',
    target_size = (128, 128),
    batch_size = 128,
    class_mode = 'binary'
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './Testing'

In [12]:
conv_base = VGG16(input_shape=(128, 128, 3), include_top = False, weights='imagenet')

58889256/58889256 [==============================] - 513s 9us/step


In [13]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [14]:
for layer in conv_base.layers:
    layer.trainable = False

In [15]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(
    train_generator, 
    steps_per_epoch = 8,
    epochs = 5,
    verbose = 1,
    validation_data = validation_generator
)

NameError: name 'train_generator' is not defined